In [1]:
cd ../src

/workspace/Script/NLP/PII/src


In [2]:
import os
import gc
import sys
import json
import time
import torch
import joblib
import random
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from pathlib import Path
import plotly.express as px
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [3]:
import re
import os
import gc
import math
import time
import json
import random
import numpy as np
import pandas as pd
import wandb

from pathlib import Path

import torch 
import torch.nn as nn
from torch.cuda import amp
import torch.optim as optim
import torch.nn.functional as F

from transformers import AutoTokenizer, AutoModel, AutoConfig
 
from data.data_utils import to_gpu,to_np
from data.dataset import FeedbackDataset,CustomCollator
from torch.utils.data import DataLoader

from model_zoo.models import FeedbackModel,span_nms,aggregate_tokens_to_words
from metrics_loss.metrics import score_feedback,score,pii_fbeta_score_v2,compute_metrics
from transformers import get_linear_schedule_with_warmup,get_cosine_schedule_with_warmup,get_polynomial_decay_schedule_with_warmup,get_cosine_with_hard_restarts_schedule_with_warmup

from sklearn.metrics import log_loss 
from tqdm.auto import tqdm

from utils.utils import count_parameters
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [4]:
os.environ["TOKENIZERS_PARALLELISM"] = "true"

# Params

In [5]:
data_path = Path(r"/database/kaggle/PII/data")
os.listdir(data_path)

['train.json',
 'dubai-ar.zip',
 'pii_dataset_fixed.csv',
 'mixtral-8x7b-v1.json',
 '.~lock.lecture2.pptx#',
 'openaddr-collected-global.zip',
 'lecture2.pptx',
 'openaddr-collected-us_west-sa.zip',
 'test.json',
 'moredata_dataset_fixed.csv',
 'sample_submission.csv']

In [6]:
CHECKPOINT_PATH = Path(r"/database/kaggle/PII/checkpoint")

In [7]:
df = pd.read_json(data_path/'train.json')
df.shape

(6807, 5)

In [8]:
df.head(2)

,document,full_text,tokens,trailing_whitespace,labels
0,7,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
1,10,Diego Estrada\n\nDesign Thinking Assignment\n\...,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."


In [9]:
external_data = False

In [10]:
if external_data:
    print("Using external data")
    dx = pd.read_json(data_path/f'mixtral-8x7b-v1.json')
    # dx[name] = -1
    df = pd.concat([df,dx],axis=0).reset_index(drop=True)

In [11]:
df.shape

(6807, 5)

In [12]:
from sklearn.model_selection import GroupKFold,StratifiedGroupKFold,KFold,StratifiedKFold

In [13]:
df['has_label'] = (df['labels'].transform(lambda x:len([i for i in x if i!="O" ]))>0)*1
seeds = [42]
folds_names = []
for K in [5]:  
    for seed in seeds:
        mskf = StratifiedKFold(n_splits=K,shuffle=True,random_state=seed)
        name = f"fold_sk_{K}_seed_{seed}"
        df[name] = -1
        for fold, (trn_, val_) in enumerate(mskf.split(df,df['has_label'])):
            df.loc[val_, name] = fold

In [14]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [15]:
LABEL2TYPE = ('NAME_STUDENT','EMAIL','USERNAME','ID_NUM', 'PHONE_NUM','URL_PERSONAL','STREET_ADDRESS','O')
for name in LABEL2TYPE[:-1]:
    df[name] = ((df['labels'].transform(lambda x:len([i for i in x if i.split('-')[-1]==name ])))>0)*1

seeds = [42]
folds_names = []
for K in [5]:  
    for seed in seeds:
        mskf = MultilabelStratifiedKFold(n_splits=K,shuffle=True,random_state=seed)
        name = f"fold_msk_{K}_seed_{seed}"
        df[name] = -1
        for fold, (trn_, val_) in enumerate(mskf.split(df,df[list(LABEL2TYPE)[:-1]])):
            df.loc[val_, name] = fold

# Data

In [16]:
from train_utils import inference_step
from types import SimpleNamespace

In [17]:
ID_TYPE = {"0-0":0,"0-1":1,
           "1-0":2,"1-1":3,
           "2-0":4,"2-1":5,
           "3-0":6,"3-1":7,
           "4-0":8,"4-1":9,
           "5-0":10,"5-1":11,
           "6-0":12,"6-1":13
          }
ID_NAME = {"0-0":"B-NAME_STUDENT","0-1":"I-NAME_STUDENT",
           "1-0":"B-EMAIL","1-1":"I-EMAIL",
           "2-0":"B-USERNAME","2-1":"I-USERNAME",
           "3-0":"B-ID_NUM","3-1":"I-ID_NUM",
           "4-0":"B-PHONE_NUM","4-1":"I-PHONE_NUM",
           "5-0":"B-URL_PERSONAL","5-1":"I-URL_PERSONAL",
           "6-0":"B-STREET_ADDRESS","6-1":"I-STREET_ADDRESS",
           "7-0":"O","7-1":"O"
          }

def inference_steps(df,folder,bs=1,fold=0):
    
    # ==== Loading Args =========== #
    f = open(f'{folder}/params.json')
    args = json.load(f)
    args = SimpleNamespace(**args)
    args.val_loader['batch_size'] = bs
    args.model['pretrained_tokenizer'] = f"{folder}/tokenizer"
    args.model['model_params']['config_path'] = f"{folder}/config.pth"
    args.model['pretrained_weights'] = None
    args.model["model_params"]['pretrained_path'] = None
#     args.model["model_params"]['max_len'] = 3048
    
    args.device = 1
    f.close()
    device = torch.device(f"cuda:{args.device}" if torch.cuda.is_available() else "cpu")
    
    # ==== Loading dataset =========== #
    tokenizer = AutoTokenizer.from_pretrained(args.model["model_params"]['model_name'])
    valid_dataset = eval(args.dataset)(df,tokenizer,**args.data["params_valid"])
    
    
    
    # ==== Loading checkpoints =========== #
    checkpoints = [x.as_posix() for x in (Path(folder)).glob("*.pth") if f"fold_{fold}" in x.as_posix()]
    print(checkpoints)
    weights = [1/len(checkpoints)]*len(checkpoints)
    
    
    # ==== Loop Inference =========== #
    doc_ids = []
    tokens = []
    tokens_v = []
    predictions = None
    gt_df = []
    for j,(checkpoint,weight) in enumerate(zip(checkpoints,weights)):
        net = FeedbackModel(**args.model["model_params"])
        net.load_state_dict(torch.load(checkpoint, map_location=lambda storage, loc: storage))
        net = net.to(device)
        net.eval()
        
        collator = CustomCollator(tokenizer,net)
        val_loader = DataLoader(valid_dataset,**args.val_loader,collate_fn=collator)
    

        
        preds = []
        with torch.no_grad():
            for data in tqdm(val_loader):
                data = to_gpu(data, device)
                
                pred = net(data)['pred']
                preds.append(pred.detach().cpu().to(torch.float32))
#                 pred  = pred.softmax(-1)
                
                
                if j==0:
                
                    doc_ids+=[data['text_id']]*pred.shape[0]
                    tokens+=np.arange(pred.shape[0]).tolist()
                    tokens_v += data['tokens']
                    data = to_np(data)
                    gt = pd.DataFrame({
                                      "document":data['text_id'],
                                      "token":np.arange(pred.shape[0]),
                                      "label":data["gt_spans"][:,1],
                                      "I":data["gt_spans"][:,2],
                                     })
                    gt_df.append(gt)

        
        if predictions is not None:
#             predictions = torch.max(predictions,torch.cat(preds,dim=0))
            predictions+= torch.cat(preds,dim=0)*weight
        else:
            predictions = torch.cat(preds,dim=0)*weight
#             predictions+= torch.cat(preds,dim=0)*weight
        print(predictions.shape)
    
    predictions = predictions.softmax(-1)
    s,i = predictions.max(-1)
    pred_df = pd.DataFrame({"document":doc_ids,
                                 "token" : tokens,
                                 "tokens":tokens_v,
                                 "label" : i.numpy() ,
                                 "score" : s.numpy() ,
#                                  "o_score":predictions[:,-1].numpy()
                                 })
    
    # ==== Loop Inference =========== #
    del valid_dataset
    del val_loader
    del net
    del s,i
    del predictions

    gc.collect()
#     torch.cuda.empty_cache()
    
    # ==== Loop Inference =========== #
#     pred_df = pred_df[(pred_df.label!=7) & (pred_df.score>0.5)].reset_index(drop=True)
#     pred_df["I"] = ((pred_df.groupby('document')['label'].transform(lambda x:x.diff())==0) & (pred_df.groupby('document')['token'].transform(lambda x:x.diff())==1))*1
#     pred_df['labels'] = pred_df['label'].astype(str)+'-'+pred_df['I'].astype(str)
#     pred_df["label_pred"] = pred_df["labels"].map(ID_TYPE).fillna(0).astype(int)
#     pred_df['row_id'] = np.arange(len(pred_df))
    

    gt_df = pd.concat(gt_df,axis=0).reset_index(drop=True)
    gt_df = gt_df[gt_df.label!=7].reset_index(drop=True)
    gt_df['labels'] = gt_df['label'].astype(str)+'-'+gt_df['I'].astype(str)
    gt_df["label_gt"] = gt_df["labels"].map(ID_TYPE).fillna(0).astype(int)
    gt_df['row_id'] = np.arange(len(gt_df))

    
    
    return pred_df , gt_df

In [18]:
FOLD_NAME = "fold_msk_5_seed_42"
model_name = "deberta-v3-large"
exp_name = "2024-02-04--vsub1"
folder = str(CHECKPOINT_PATH/Path(fr'{FOLD_NAME}/{model_name}/{exp_name}')) 
os.listdir(folder)

['fold_3 _epoch_5 _step_5856 _valid_loss_0.0071 _f5_micro_0.9985 _train_loss_0.0006.pth',
 'tokenizer',
 'params.json',
 'fold_2 _epoch_9 _step_7331 _valid_loss_0.0072 _f5_micro_0.9984 _train_loss_0.0001.pth',
 'fold_1 _epoch_5 _step_4397 _valid_loss_0.0060 _f5_micro_0.9982 _train_loss_0.0012.pth',
 'fold_4 _epoch_5 _step_7322 _valid_loss_0.0051 _f5_micro_0.9978 _train_loss_0.0011.pth',
 'config.pth',
 'fold_0 _epoch_8 _step_7331 _valid_loss_0.0080 _f5_micro_0.9981 _train_loss_0.0004.pth']

In [19]:
df[FOLD_NAME].value_counts()

4    1362
2    1362
3    1361
0    1361
1    1361
Name: fold_msk_5_seed_42, dtype: int64

In [20]:
FOLD = 0

In [21]:
pred_df_dv3,gt_df = inference_steps(df[df[FOLD_NAME]==FOLD],folder,bs=1,fold=FOLD)

Loaded 1361 samples.
['/database/kaggle/PII/checkpoint/fold_msk_5_seed_42/deberta-v3-large/2024-02-04--vsub1/fold_0 _epoch_8 _step_7331 _valid_loss_0.0080 _f5_micro_0.9981 _train_loss_0.0004.pth']
Pooling: MeanPooling


  0%|          | 0/1361 [00:01<?, ?it/s]

torch.Size([997332, 8])


In [22]:
g  = gt_df.copy()

In [23]:
gt_df['label'] = gt_df['labels'].map(ID_NAME)

In [62]:
pred_df = pred_df_dv3.copy()#pred_df_dv3[(pred_df_dv3.label!=7) & (pred_df_dv3.score>0.15)].reset_index(drop=True)

In [63]:
def make_pred_df(pred_df,threshold=0.15):
    
    pred_df["label_next_e_prev"] = pred_df.groupby('document')['label'].transform(lambda x: (x.shift(1)==x.shift(-1))*1)
    pred_df["label_next"] = pred_df.groupby('document')['label'].transform(lambda x: x.shift(1))
    pred_df["label_next_e_prev"] = ((pred_df["label_next_e_prev"]==1) & (pred_df["label_next"]==6))*1
    pred_df["score_next"] = pred_df.groupby('document')['score'].transform(lambda x: x.shift(1))
    pred_df.loc[pred_df["label_next_e_prev"]==1,"label"] = pred_df.loc[pred_df["label_next_e_prev"]==1,"label_next"]
    pred_df.loc[pred_df["label_next_e_prev"]==1,"score"] = pred_df.loc[pred_df["label_next_e_prev"]==1,"score_next"]
    
    
    pred_df = pred_df[(pred_df.label!=7) & ((pred_df.score>threshold))].reset_index(drop=True)
    
    pred_df = pred_df[~((pred_df.label==0) & ((pred_df.token_size<=1)))].reset_index(drop=True)
    pred_df["I"] = ((pred_df.groupby('document')['label'].transform(lambda x:x.diff())==0) & (pred_df.groupby('document')['token'].transform(lambda x:x.diff())==1))*1
    pred_df['labels'] = pred_df['label'].astype(str)+'-'+pred_df['I'].astype(str)
    pred_df["label_pred"] = pred_df["labels"].map(ID_TYPE).fillna(0).astype(int)
    pred_df['row_id'] = np.arange(len(pred_df))
    
    pred_df['label'] = pred_df['labels'].map(ID_NAME)
    return pred_df

In [64]:
pred_df = make_pred_df(pred_df,threshold=0.15)

AttributeError: 'DataFrame' object has no attribute 'token_size'

In [27]:
s = compute_metrics(pred_df, gt_df)
s

{'f5_prec': 0.7753623188405797,
 'f5_rec': 0.9907407407407407,
 'f5_micro': 0.9802677942212826,
 'ents_per_type': {'NAME_STUDENT': 0.9830722986869167,
  'URL_PERSONAL': 0.989345509893455,
  'PHONE_NUM': 1.0,
  'ID_NUM': 0.8965517241379312,
  'USERNAME': 0.9285714285714286,
  'EMAIL': 1.0}}

In [33]:
pred_df['token_size'] = pred_df['tokens'].transform(len)

In [35]:
pred_df['token_size'].value_counts()

5     170
6     129
4      92
7      85
3      55
8      53
9      23
12     13
2       9
43      9
16      6
10      6
15      5
1       4
41      4
38      4
18      2
45      2
50      2
11      2
36      2
19      2
53      1
28      1
21      1
13      1
17      1
74      1
37      1
39      1
42      1
67      1
47      1
Name: token_size, dtype: int64

In [36]:
pred_df.shape,pred_df[pred_df.token_size==1].shape

((690, 13), (4, 13))

In [37]:
pred_df[pred_df.token_size==1]

,document,token,tokens,label,score,label_next_e_prev,label_next,score_next,I,labels,label_pred,row_id,token_size
103,4381,18,(,B-PHONE_NUM,0.9999,0,0.0000,0.1250,0,4-0,8,103,1
105,4381,20,-,I-PHONE_NUM,1.0000,0,4.0000,1.0000,1,4-1,9,105,1
141,5001,1609,I,I-NAME_STUDENT,0.9969,0,0.0000,0.9995,1,0-1,1,141,1
634,15720,573,.,I-ID_NUM,0.8217,0,3.0000,0.9307,1,3-1,7,634,1


In [36]:
s = compute_metrics(pred_df, gt_df)
s

{'f5_prec': 0.7753623188405797,
 'f5_rec': 0.9907407407407407,
 'f5_micro': 0.9802677942212826,
 'ents_per_type': {'NAME_STUDENT': 0.9830722986869167,
  'ID_NUM': 0.8965517241379312,
  'URL_PERSONAL': 0.989345509893455,
  'EMAIL': 1.0,
  'USERNAME': 0.9285714285714286,
  'PHONE_NUM': 1.0}}

In [37]:
s['ents_per_type']

{'NAME_STUDENT': 0.9830722986869167,
 'ID_NUM': 0.8965517241379312,
 'URL_PERSONAL': 0.989345509893455,
 'EMAIL': 1.0,
 'USERNAME': 0.9285714285714286,
 'PHONE_NUM': 1.0}

In [38]:
pd.DataFrame({x:[y] for x,y in s['ents_per_type'].items()})

,NAME_STUDENT,ID_NUM,URL_PERSONAL,EMAIL,USERNAME,PHONE_NUM
0,0.9831,0.8966,0.9893,1.0000,0.9286,1.0000


In [50]:
documents = df[df[FOLD_NAME]==FOLD].document.unique()
len(documents)

1361

In [51]:
df_score = []
for doc in tqdm(documents):
    p = pred_df[pred_df.document==doc].reset_index(drop=True)
    gp = gt_df[gt_df.document==doc].reset_index(drop=True)
    
    s = compute_metrics(p, gp)
    
    d = pd.DataFrame({x:[y] for x,y in s['ents_per_type'].items()})
    d["f5_micro"] = s['f5_micro']
    d['document'] = doc
    df_score.append(d)

  0%|          | 0/1361 [00:00<?, ?it/s]

In [52]:
gp

,document,token,label,I,labels,label_gt,row_id


In [53]:
df_score = pd.concat(df_score).reset_index(drop=True)

In [54]:
df_score[df_score.f5_micro==0]

,NAME_STUDENT,f5_micro,document,URL_PERSONAL,ID_NUM,PHONE_NUM,EMAIL,USERNAME
4,0.0000,0.0000,204,NaN,NaN,NaN,NaN,NaN
122,0.0000,0.0000,8758,NaN,NaN,NaN,NaN,NaN
127,0.0000,0.0000,9031,NaN,NaN,NaN,NaN,NaN
132,0.0000,0.0000,9313,NaN,NaN,NaN,NaN,NaN
145,0.0000,0.0000,9961,NaN,NaN,NaN,NaN,NaN
149,0.0000,0.0000,10070,NaN,NaN,NaN,NaN,NaN
151,0.0000,0.0000,10166,NaN,NaN,NaN,NaN,NaN
171,0.0000,0.0000,11856,NaN,NaN,NaN,NaN,NaN
174,0.0000,0.0000,11896,NaN,NaN,NaN,NaN,NaN
175,0.0000,0.0000,11901,NaN,NaN,NaN,NaN,0.0000


In [41]:
doc = 5001

In [42]:
dx = pred_df.groupby("document")['label'].agg(list).reset_index()

In [43]:
dx

,document,label
0,16,"[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
1,56,"[O, O, B-NAME_STUDENT, O, B-NAME_STUDENT, O, B..."
2,112,"[O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT..."
3,166,"[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."
4,204,"[O, O, O, B-NAME_STUDENT, O, O, O, B-NAME_STUD..."
...,...,...
1356,22637,"[O, O, O, O, O, O, O, O, O, O, O, B-NAME_STUDE..."
1357,22646,"[O, O, O, O, B-NAME_STUDENT, O, B-NAME_STUDENT..."
1358,22652,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1359,22664,"[O, O, O, O, O, B-NAME_STUDENT, O, O, O, O, O,..."


In [44]:
dfold = df[df[FOLD_NAME]==FOLD].reset_index(drop=True)
dfold.shape

(1361, 15)

In [45]:
dfold = dfold.merge(dx,how='left',on='document')

In [46]:
dfold.head()

,document,full_text,tokens,trailing_whitespace,labels,has_label,fold_sk_5_seed_42,NAME_STUDENT,EMAIL,USERNAME,ID_NUM,PHONE_NUM,URL_PERSONAL,STREET_ADDRESS,fold_msk_5_seed_42,label
0,16,Reporting process\n\nby Gilberto Gamboa\n\nCha...,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O...",1,1,1,0,0,0,0,0,0,0,"[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
1,56,Assignment: Visualization Reflection Submitt...,"[Assignment, :, , Visualization, , Reflecti...","[False, False, False, False, False, False, Fal...","[O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST...",1,0,1,0,0,0,0,0,0,0,"[O, O, B-NAME_STUDENT, O, B-NAME_STUDENT, O, B..."
2,112,Reflection – Learning Launch\n\nFrancisco Ferr...,"[Reflection, –, Learning, Launch, \n\n, Franci...","[True, True, True, False, False, True, False, ...","[O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT...",1,2,1,0,0,0,0,0,0,0,"[O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT..."
3,166,Pepa Medrano\n\nDesign Thinking for Innovation...,"[Pepa, Medrano, \n\n, Design, Thinking, for, I...","[True, False, False, True, True, True, True, F...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O...",1,2,1,0,0,0,0,0,0,0,"[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."
4,204,Reflection – Visualization Deiby\n\nChalleng...,"[Reflection, –, Visualization, , Deiby, \n\n...","[True, True, True, False, False, False, True, ...","[O, O, O, O, B-NAME_STUDENT, O, O, O, O, O, O,...",1,2,1,0,0,0,0,0,0,0,"[O, O, O, B-NAME_STUDENT, O, O, O, B-NAME_STUD..."


In [47]:
dfold['len_tok'] = dfold['labels'].transform(lambda x:len(x))
dfold['len_tok_p'] = dfold['label'].transform(lambda x:len(x))

In [48]:
(dfold['len_tok']==dfold['len_tok_p']).value_counts()

True    1361
dtype: int64

In [49]:
from data.data_utils import get_offset_mapping

In [50]:
import spacy
from spacy import displacy
from pylab import cm, matplotlib
import os

colors = {
            'NAME_STUDENT': '#8000ff',
            'EMAIL': '#2b7ff6',
            'USERNAME': '#2adddd',
            'ID_NUM': '#80ffb4',
            'PHONE_NUM': 'd4dd80',
            'URL_PERSONAL': '#ff8042',
            'STREET_ADDRESS': '#ff0000'
         }


def visualize(full_text,offset_mapping,labels):
    
    ents = []
    for offset,lab in zip(offset_mapping,labels):
        ents.append({
                        'start': int(offset[0]), 
                         'end': int(offset[1]), 
                         'label': str(lab.split('-')[1]) #+ ' - ' + str(row['discourse_effectiveness'])
                    })

    doc2 = {
        "text": full_text,
        "ents": ents,
#         "title": "idx"
    }

    options = {"ents": list(colors.keys()), "colors": colors}
    displacy.render(doc2, style="ent", options=options, manual=True, jupyter=True)

In [51]:
idx = random.choice(dfold[dfold.NAME_STUDENT>0].index)
idx = dfold[dfold.document==doc].index[0]
# Example usage:
# idx = "uzvemcjmik"
full_text_ds = dfold.iloc[idx]['full_text']
tokens_ds = dfold.iloc[idx]['tokens']
labels_ds = dfold.iloc[idx]['labels']
labels = dfold.iloc[idx]['label']
idx,dfold.iloc[idx]['document']

(50, 5001)

In [52]:
offset_mapping = get_offset_mapping(full_text_ds, tokens_ds)
offset_mapping_ = [x for (x,y) in zip(offset_mapping,labels_ds) if y!="O"]
labels_ = [x for x in labels_ds if x!="O"]

In [53]:
visualize(full_text_ds,offset_mapping_,labels_)

In [54]:
offset_mapping = get_offset_mapping(full_text_ds, tokens_ds)
offset_mapping_ = [x for (x,y) in zip(offset_mapping,labels) if y!="O"]
labels_ = [x for x in labels_ds if x!="O"]

In [55]:
visualize(full_text_ds,offset_mapping_,labels_)

In [56]:
# doc = 204
x = pred_df[pred_df.document==doc]

In [98]:
x['tokens'] = tokens_ds

In [57]:
x['label_gt'] = labels_ds

In [131]:
x['label_e'] = x['label']

In [123]:
x.loc[x.label_e.shift(-1)==x.label_e.shift(1),'label_e'] = x.loc[x.label_e.shift(-1)==x.label_e.shift(1),'label_e'].shift(1)

In [132]:
x.label.value_counts()

O                 851
B-NAME_STUDENT     61
Name: label, dtype: int64

In [61]:
x[x.label!="O"]

,document,token,tokens,label,score,label_next_e_prev,label_next,score_next,I,labels,label_pred,row_id,label_gt
42619,5001,34,Rya,B-NAME_STUDENT,0.9701,0,7.0000,1.0000,0,0-0,0,42619,O
42643,5001,58,,B-NAME_STUDENT,0.1250,0,7.0000,1.0000,0,0-0,0,42643,O
42666,5001,81,,B-NAME_STUDENT,0.1250,0,7.0000,1.0000,0,0-0,0,42666,O
42681,5001,96,,B-NAME_STUDENT,0.1250,0,7.0000,1.0000,0,0-0,0,42681,O
42704,5001,119,,B-NAME_STUDENT,0.1250,0,7.0000,1.0000,0,0-0,0,42704,O
42725,5001,140,,B-NAME_STUDENT,0.1250,0,7.0000,1.0000,0,0-0,0,42725,O
42767,5001,182,,B-NAME_STUDENT,0.1250,0,7.0000,1.0000,0,0-0,0,42767,O
42794,5001,209,,B-NAME_STUDENT,0.1250,0,7.0000,1.0000,0,0-0,0,42794,O
42839,5001,254,,B-NAME_STUDENT,0.1250,0,7.0000,1.0000,0,0-0,0,42839,O
42863,5001,278,,B-NAME_STUDENT,0.1250,0,7.0000,1.0000,0,0-0,0,42863,O


In [60]:
x[x(.score>0.125) & (x.label!="O")]

SyntaxError: invalid syntax (2834079789.py, line 1)